<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h1 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">M2.4 - معالجة سجلات بيانات المناخ الطويلة بصورة متزامنة</h1>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">جزء من: <a href="https://github.com/OpenClimateScience/M2-Computational-Climate-Science" dir="ltr">Computational Climate Science</a> | الدرس السابق | الدرس التالي</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>المحتويات:</strong></p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قيود الموارد في الحوسبة
  <ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
    <li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المشكلات المقيدة بالمعالج <span dir="ltr" style="unicode-bidi: isolate;">(CPU-bound)</span></li>
  </ul>
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المعالجة المتزامنة لمجموعات بيانات المناخ الكبيرة</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> باستخدام معادلة <span dir="ltr" style="unicode-bidi: isolate;">Hargreaves</span>
  <ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
    <li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب إشعاع أعلى الغلاف الجوي <span dir="ltr" style="unicode-bidi: isolate;">(TOA)</span></li>
    <li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">دوال موثقة توثيقًا جيدًا</li>
    <li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الدوال المتجهة <span dir="ltr" style="unicode-bidi: isolate;">(Vectorized)</span></li>
    <li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اشتقاق متغيرات من إحداثيات <span dir="ltr" style="unicode-bidi: isolate;">xarray</span></li>
  </ul>
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تطبيق دالة على كتل مستقلة
  <ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
    <li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إسناد/تطبيق دالة عامة عبر خريطة <span dir="ltr" style="unicode-bidi: isolate;">(Mapping)</span></li>
  </ul>
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تحليل موارد الحوسبة <span dir="ltr" style="unicode-bidi: isolate;">(Profiling)</span>
  <ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
    <li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قياس الزمن الحقيقي للمهمة باستخدام <span dir="ltr" style="unicode-bidi: isolate;">timeit</span></li>
  </ul>
</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الخلاصة</li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نظرة عامة</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في الدرس السابق ناقشنا كيف يمكن استخدام نموذج «الدلو» البسيط لقياس الفرق بين إمداد المياه (الهطول) وفقد المياه (التبخر-النتح المحتمل <span dir="ltr" style="unicode-bidi: isolate;">PET</span>). وتفيد نسبة هاتين الكميتين أيضًا كمؤشر يوضح مقدار ما يُعوَّض من فقد المياه بواسطة الهطول.</p>
$$
\text{Percentage replenished} \approx 100\times \frac{\text{Precipitation}}{\text{PET}}
$$
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">طريقة حساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> التي سنستخدمها هي طريقة <a href="https://www.fao.org/4/X0490E/x0490e07.htm#minimum%20data%20requirements" dir="ltr">Hargreaves method</a> (Allen وآخرون، 2000)، لأنها تتطلب بيانات الحرارة فقط. سنستخدم بيانات الحرارة من <span dir="ltr" style="unicode-bidi: isolate;">MERRA-2</span> لحساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span>، ثم سنستخدم بيانات الهطول من <span dir="ltr" style="unicode-bidi: isolate;">CHIRPS</span> مرة أخرى لاشتقاق مؤشر الجفاف الهيدرولوجي.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ورغم وجود مصادر عديدة جاهزة لبيانات <span dir="ltr" style="unicode-bidi: isolate;">PET</span>، سنقوم بحسابها بأنفسنا لاكتساب خبرة أكبر في التعامل مع مجموعات بيانات مناخية كبيرة. وخلال ذلك، سنتعلم كيف يمكن معالجة مجموعات البيانات الكبيرة <strong>بصورة متزامنة</strong>، بما يساعد على معالجة مشكلتين شائعتين.</p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أن مجموعة البيانات كاملة أكبر من أن تُحمَّل إلى الذاكرة دفعة واحدة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أن معالجة البيانات قد تستغرق وقتًا طويلًا، إما لأن البيانات ضخمة أو لأن الحسابات معقدة.</li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قيود الموارد في الحوسبة</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عمومًا، كلما كانت مجموعة البيانات أكبر احتجنا إلى موارد حوسبية أكثر لتحليلها. لكن نوع الموارد المطلوبة يعتمد على طبيعة البيانات ونوع التحليل الذي نريد تنفيذه.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>في مسائل الحوسبة، توجد ثلاثة أنواع رئيسية من القيود (عنق الزجاجة) التي قد تحد من تشغيل الخوارزمية:</strong></p>
<ol style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>سرعة القراءة والكتابة من نظام الملفات</strong></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>ذاكرة الحاسوب</strong></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>سرعة ساعة وحدة المعالجة المركزية</strong> <span dir="ltr" style="unicode-bidi: isolate;">(CPU)</span> مثل <span dir="ltr" style="unicode-bidi: isolate;">3 GHz</span></li>
</ol>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يحدث القيد من <strong>النوع 1</strong> عندما تكون لدينا مجموعات بيانات كبيرة جدًا أو عندما تكون سرعات القراءة/الكتابة بطيئة. تعتمد السرعة على وسيط التخزين؛ فالأقراص ذات الحالة الصلبة أسرع عادة من الأقراص الدوّارة. وإذا كان التخزين على جهاز شبكي <span dir="ltr" style="unicode-bidi: isolate;">(NAS)</span> بدل القرص المحلي، فإن سرعة الشبكة تصبح جزءًا من هذا القيد. وتسمى المشكلات التي يحدها هذا النوع <strong>مقيدة بالإدخال/الإخراج</strong> <span dir="ltr" style="unicode-bidi: isolate;">(I/O-Bound)</span>.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أما القيد من <strong>النوع 2</strong> فيظهر عندما نحاول تخزين مجموعة البيانات كاملة في الذاكرة أو عندما ينتج التحليل حجمًا كبيرًا من البيانات داخل الذاكرة. الذاكرة محدودة، والانتقال القسري لجزء من البيانات إلى القرص يسمى <span dir="ltr" style="unicode-bidi: isolate;">swapping</span> وهو بطيء جدًا. لذلك قد لا يتوقف البرنامج، لكنه يصبح بطيئًا للغاية. وتسمى هذه المشكلات <strong>مقيدة بالذاكرة</strong> <span dir="ltr" style="unicode-bidi: isolate;">(Memory-Bound)</span>.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">القيد من <strong>النوع 3</strong> يرتبط بالخوارزمية أكثر من ارتباطه بالبيانات. على سبيل المثال، إذا كنا نقرأ بيانات ضخمة ثم نجري تحويل وحدة بسيطًا (مثل ضرب القيم في <span dir="ltr" style="unicode-bidi: isolate;">1000</span> ثم حفظها)، فسرعة <span dir="ltr" style="unicode-bidi: isolate;">CPU</span> قد لا تكون مشكلة كبيرة، لأن ضرب الأعداد سريع جدًا. لكن السرعة الفعلية تعتمد على سرعة <span dir="ltr" style="unicode-bidi: isolate;">CPU</span>. وتسمى هذه المشكلات <strong>مقيدة بالمعالج</strong> <span dir="ltr" style="unicode-bidi: isolate;">(CPU-Bound)</span>.</p>
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المشكلات المقيدة بالمعالج <span dir="ltr" style="unicode-bidi: isolate;">(CPU-bound)</span></h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تاريخيًا، حظي النوع 3 باهتمام كبير. فقد أدت تحسينات تصنيع المعالجات إلى شرائح أسرع فأسرع. وكان Gordon Moore من أوائل من لاحظ اتجاه هذا التسارع، وأصبحت «قانون مور» <span dir="ltr" style="unicode-bidi: isolate;">(Moore's Law)</span> فكرة محورية في الصناعة لوقت طويل؛ وهي ميل سرعة ساعة المعالج إلى التضاعف كل <span dir="ltr" style="unicode-bidi: isolate;">2</span> سنة (Moore، 1965).</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لكن توجد إشارات حديثة إلى أن هذا المعدل قد يتباطأ. يمكن الرجوع إلى هذا المقال للاطلاع: <a href="https://www.tomshardware.com/tech-industry/semiconductors/intels-ceo-says-moores-law-is-slowing-to-a-three-year-cadence-but-its-not-dead-yet" dir="ltr">Tom's Hardware</a>. ومن الأسباب المهمة مشكلة تبديد الحرارة. إذ إن تصغير الترانزستورات يزيد سخونتها مع مرور التيار الكهربائي، لذلك يركز تصميم الشرائح الحديثة كثيرًا على منع ارتفاع الحرارة.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ومع ذلك، عند تجميع عدة معالجات منخفضة الاستهلاك يمكن الحصول على أداء أفضل من معالج واحد عالي الاستهلاك. يوضح الشكل التالي فكرة المعالجة التسلسلية والمتزامنة والمتوازية.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><img src="./assets/M2_concurrency.jpg" alt="M2_concurrency" /></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وصف الصورة: صورة من <a href="https://kwahome.medium.com/concurrency-is-not-parallelism-a5451d1cde8d" dir="ltr">Kevin Wahome</a>.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في أسلوب <strong>المعالجة المتزامنة</strong> <span dir="ltr" style="unicode-bidi: isolate;">(Concurrency)</span> يمكن للحاسوب التبديل بين المهام بسرعة تجعلها تبدو وكأنها تُنفَّذ في الوقت نفسه عبر «خيوط» <span dir="ltr" style="unicode-bidi: isolate;">(Threads)</span>. أما أسلوب <strong>المعالجة المتوازية</strong> <span dir="ltr" style="unicode-bidi: isolate;">(Parallel)</span> فيستخدم عدة وحدات معالجة لتنفيذ مهام مستقلة بالتزامن فعليًا.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>اليوم سنرى كيف يمكن استخدام عدة وحدات معالجة لتجزئة المشكلة إلى أجزاء أصغر تُنفَّذ بالتزامن.</strong> وقد تجعل بعض الأدوات الفرق بين المتزامن والمتوازي غير واضح عمليًا، لذا سنستخدم في هذا الدرس مصطلح «المعالجة المتزامنة» للإشارة إلى كلا الأسلوبين.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المعالجة المتزامنة لمجموعات بيانات المناخ الكبيرة</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كما رأينا سابقًا، يمكننا استخدام <span dir="ltr" style="unicode-bidi: isolate;">earthaccess</span> لتنزيل بيانات <a href="https://gmao.gsfc.nasa.gov/reanalysis/MERRA-2/" dir="ltr">MERRA-2</a> من <span dir="ltr" style="unicode-bidi: isolate;">NASA EarthData Search</span>. سنستخدم البيانات اليومية المجمعة التي استخدمناها سابقًا، مع قيمة <span dir="ltr" style="unicode-bidi: isolate;">short_name</span> التالية: <span dir="ltr" style="unicode-bidi: isolate;">"M2SDNXSLV"</span>.</p>
</div>

In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2024-01-01", "2024-05-31"))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أفضل ممارسة</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تذكّر:</strong> نريد التأكد من عدم تعديل البيانات الخام بالخطأ، لذا يجب تنزيل هذه البيانات إلى مجلد مخصص للبيانات الخام.</p>
</div>

In [ ]:
# Could take about 1 minute on a broadband connection
earthaccess.download(results, 'data_raw/MERRA2')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مرة أخرى سنستخدم <span dir="ltr" style="unicode-bidi: isolate;">xr.open_mfdataset()</span> لفتح مجموعة الملفات لدينا على أنها <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> واحد.</p>
</div>

In [ ]:
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4')
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">متغيرات بيانات <span dir="ltr" style="unicode-bidi: isolate;">MERRA-2</span> التي نهتم بها هي:</p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">T2MMAX</span>، درجة الحرارة اليومية العظمى (درجات مئوية).</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">T2MMEAN</span>، متوسط درجة الحرارة اليومية (درجات مئوية).</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><span dir="ltr" style="unicode-bidi: isolate;">T2MMIN</span>، درجة الحرارة اليومية الصغرى (درجات مئوية).</li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ أن لدينا <span dir="ltr" style="unicode-bidi: isolate;">122</span> يومًا من البيانات، لذا يمتلك مكعب البيانات محور زمن بطول <span dir="ltr" style="unicode-bidi: isolate;">122</span> خطوة زمنية يومية. كما قام <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> تلقائيًا بتقسيم مجموعة البيانات إلى <strong>كتل</strong> <span dir="ltr" style="unicode-bidi: isolate;">(chunks)</span> متساوية الحجم يمكن معالجتها بشكل مستقل.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ملاحظة: في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>، الكتلة <span dir="ltr" style="unicode-bidi: isolate;">(chunk)</span> (وتسمى أيضًا كتلة/بلوك <span dir="ltr" style="unicode-bidi: isolate;">block</span>) هي جزء محدد من البيانات عبر محور واحد أو أكثر.</p>
</div>

In [ ]:
ds['T2MMEAN']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>حجم الكتل وشكلها مهمان إذا كنا سنستخدم المعالجة المتزامنة.</strong> على سبيل المثال، إذا أردنا حساب اتجاهات طويلة الأمد، فلن نستطيع ذلك باستخدام التقسيم الحالي لأن كل كتلة تحتوي خطوة زمنية واحدة فقط.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا محاولة استخدام معامل <span dir="ltr" style="unicode-bidi: isolate;">chunks</span> في <span dir="ltr" style="unicode-bidi: isolate;">open_mfdataset()</span> لتحديد أن تحتوي الكتل على <span dir="ltr" style="unicode-bidi: isolate;">122</span> عنصرًا على محور <span dir="ltr" style="unicode-bidi: isolate;">time</span>، مثلًا.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن الرجوع إلى توثيق <a href="https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html" dir="ltr">xarray.open_mfdataset</a>.</p>
</div>

In [ ]:
# The "chunks" argument tells xarray what size the chunks should be on one or more axes
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4', chunks = {'time': 122})
ds['T2MMEAN'].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لكن من الواضح أن ذلك لم ينجح؛ فما زالت كل كتلة تحتوي خطوة زمنية واحدة.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تنبيه</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>السبب هو أن معامل <span dir="ltr" style="unicode-bidi: isolate;">chunks</span> يُقيَّم بصورة مستقلة لكل ملف.</strong> تقوم <span dir="ltr" style="unicode-bidi: isolate;">xr.open_mfdataset()</span> بفتح عدة ملفات ودمجها في مجموعة بيانات واحدة، لكن بما أن كل ملف يمثل خطوة زمنية مختلفة، فلا يمكن إنشاء كتل تمتد عبر ملفات متعددة في هذه الحالة.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بدلًا من ذلك، يمكننا تحديد حجم الكتل على محوري <span dir="ltr" style="unicode-bidi: isolate;">lat</span> و<span dir="ltr" style="unicode-bidi: isolate;">lon</span> لأن ذلك لا يتطلب الامتداد عبر ملفات متعددة. أدناه نحدد أحجامًا تجعل لدينا <span dir="ltr" style="unicode-bidi: isolate;">4</span> كتل لكل ملف.</p>
</div>

In [ ]:
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4', chunks = {'lat': 182, 'lon': 288})
ds['T2MMEAN'].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>إذا احتجنا فعليًا إلى أن تحتوي كل كتلة على محور <span dir="ltr" style="unicode-bidi: isolate;">time</span> كاملًا (عدد <span dir="ltr" style="unicode-bidi: isolate;">122</span> خطوة زمنية)، فعلينا إعادة تقسيم البيانات إلى كتل بعد قراءة جميع الملفات.</strong> يمكننا فعل ذلك باستخدام دالة <a href="https://docs.xarray.dev/en/stable/generated/xarray.DataArray.chunk.html" dir="ltr">xarray.DataArray.chunk</a> أو ما يكافئها على <span dir="ltr" style="unicode-bidi: isolate;">Dataset</span>.</p>
</div>

In [ ]:
# TODO Re-chunking the data *after* loading is generally inefficient, but might be necessary; 
#    give example of "what if" we were interested in calculating trends

ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4')
ds = ds.chunk({'time': 122})
ds['T2MMEAN'].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أفضل ممارسة</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عمومًا من الأفضل استخدام معامل <span dir="ltr" style="unicode-bidi: isolate;">chunks</span> لأن إعادة تقسيم البيانات بعد التحميل غير فعالة. لكن عندما تحتاج الكتل إلى الامتداد عبر ملفات متعددة، ستضطر لإعادة التقسيم باستخدام <span dir="ltr" style="unicode-bidi: isolate;">chunk()</span>.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في هذه الحالة نحن لا نحتاج فعليًا أن تحتوي الكتل على <span dir="ltr" style="unicode-bidi: isolate;">122</span> خطوة زمنية. يمكننا قبول التقسيم الافتراضي الذي يختاره <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>. وإذا استخدمنا <span dir="ltr" style="unicode-bidi: isolate;">chunks = 'auto'</span> فسيختار <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> تحميل كل ملف ككتلة مستقلة، أي توجد كتلة واحدة لكل ملف.</p>
</div>

In [ ]:
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4', chunks = 'auto')
ds['T2MMEAN'].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاحظ مدى سرعة تنفيذ كتل الشيفرة أعلاه. السبب هو أن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لم يحمّل أي بيانات إلى الذاكرة بعد. هل تتذكر <strong>التقييم الكسول</strong> <span dir="ltr" style="unicode-bidi: isolate;">(Lazy evaluation)</span>؟ لن يحمّل <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> البيانات إلى الذاكرة إلا عندما يصبح ذلك ضروريًا. كما أن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> لا يحتاج لتحميل البيانات ليعرض لنا معلومات «التمثيل»؛ يكفيه قراءة جزء صغير من كل ملف لمعرفة كيفية تمثيل المجموعة كاملة.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> باستخدام معادلة <span dir="ltr" style="unicode-bidi: isolate;">Hargreaves</span></h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لكي نحسب نسبة الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">PET</span>، نحتاج أولًا إلى حساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> باستخدام معادلة <span dir="ltr" style="unicode-bidi: isolate;">Hargreaves</span>:</p>
$$
\text{PET} = 0.0023 \times R_A \times \sqrt{T_{max} - T_{min}} \times (T + 17.8)
$$
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في المعادلة أعلاه، يمثل $R_A$ إشعاع الشمس أعلى الغلاف الجوي <span dir="ltr" style="unicode-bidi: isolate;">(TOA)</span>، وتمثل $T$ و$T_{max}$ و$T_{min}$ درجات الحرارة: المتوسطة والعظمى والصغرى على الترتيب.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أفضل ممارسة</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">معادلة <span dir="ltr" style="unicode-bidi: isolate;">Hargreaves</span> معقدة بما يكفي لتتطلب عدة خطوات معالجة للوصول إلى هدفنا (نسبة الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">PET</span> لمنطقة محددة). هذا يتطلب الانتباه لمزالق شائعة في علم البيانات الحاسوبي.</p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ضمان تنفيذ الخطوات بالترتيب الصحيح حتى تُهيأ هياكل البيانات والمتغيرات بصورة سليمة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">التأكد من صحة وحدات القياس وتوافقها عبر الخطوات المختلفة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">توثيق كل خطوة لتسهيل كشف الأخطاء وتمكين طرف ثالث من التحقق أو إعادة الإنتاج.</li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن أن تساعدنا تقنية «التفكيك» <span dir="ltr" style="unicode-bidi: isolate;">(Decomposition)</span> على التخطيط، وذلك بتقسيم المشكلة إلى خطوات مستقلة قابلة للإدارة:</p>
<ol style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تحميل مدخلات بيانات الحرارة المطلوبة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب إشعاع أعلى الغلاف الجوي <span dir="ltr" style="unicode-bidi: isolate;">(TOA)</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> باستخدام معادلة <span dir="ltr" style="unicode-bidi: isolate;">Hargreaves</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب نسبة الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">PET</span>.</li>
</ol>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>هذه الخطوات المرتبة تساعدنا على تنظيم سير العمل بصورة واضحة للآخرين.</strong> لقد حمّلنا بيانات الحرارة بالفعل (الخطوة 1)، فلننتقل إلى حساب إشعاع <span dir="ltr" style="unicode-bidi: isolate;">TOA</span>.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">حساب إشعاع أعلى الغلاف الجوي <span dir="ltr" style="unicode-bidi: isolate;">(TOA)</span></h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">فيما يلي دالة لحساب إشعاع <span dir="ltr" style="unicode-bidi: isolate;">TOA</span> اعتمادًا على إرشادات <span dir="ltr" style="unicode-bidi: isolate;">FAO</span>. يمكن الرجوع إلى هذا الرابط: <a href="https://www.fao.org/4/X0490E/x0490e07.htm#radiation" dir="ltr">FAO radiation guidance</a>.</p>
</div>

In [ ]:
import numpy as np

def toa_radiation(latitude, doy):
    '''
    Top-of-atmosphere (TOA) radiation for a given latitude (L) and day of year
    (DOY) can be calculated as:

    R = ((24 * 60) / pi) * G * d * (w * sin(L) * sin(D) + cos(L) * cos(D) * sin(w))

    Where G is the solar constant, 0.0820 [MJ m-2 day-1]; d is the (inverse) 
    relative earth-sun distance; w is the sunset hour angle; and D is the solar
    declination angle.
    
    For more information, consult the FAO documentation:

        https://www.fao.org/4/X0490E/x0490e07.htm#radiation
    
    Parameters
    ----------
    latitude : float
        The latitude on earth, in degrees, where southern latitudes
        are represented as negative numbers
    doy : int
        The day of the year (DOY), an integer on [1,366]
    
    Returns
    -------
    Number
        Top-of-atmosphere (TOA) radiation, in [MJ m-2 day-1]
    '''
    assert isinstance(doy, int) or issubclass(doy.dtype.type, np.integer), 'The "doy" argument must be an integer'
    assert np.all(doy >= 1) and np.all(doy <= 366), 'The "doy" argument must be between 1 and 366, inclusive'
    
    solar_constant = 0.0820 # [MJ m-2 day-1]
    pi = 3.14159
    
    # Convert latitude from degrees to radians
    latitude_radians = np.deg2rad(latitude)
    # Inverse Earth-Sun distance (relative), as a function of day-of-year (DOY)
    earth_sun_dist = 1 + 0.0033 * np.cos((doy * 2 * pi) / 365)
    # Solar declination, as a function of DOY
    declination = 0.409 * np.sin(((doy * 2 * pi) / 365) - 1.39)
    
    # Sunset hour angle; we use np.where() below to guard against
    #   warnings where arccos() would return invalid values, which
    #   happens when the argument is outside [-1, 1]
    _hour_angle = -np.tan(latitude_radians) * np.tan(declination)
    _hour_angle = np.where(np.abs(_hour_angle) > 1, np.nan, _hour_angle)
    sunset_hour_angle = np.arccos(_hour_angle)

    # Incident radiation, depends only on the relative earth-sun distance
    inc_radiation = ((24 * 60) / pi) * solar_constant * earth_sun_dist
    return inc_radiation * (sunset_hour_angle * np.sin(latitude_radians) * np.sin(declination) +
            np.cos(latitude_radians) * np.cos(declination) * np.sin(sunset_hour_angle))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">دوال موثقة توثيقًا جيدًا</h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>هناك عدة نقاط مهمّة في هذه الدالة.</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يوجد <span dir="ltr" style="unicode-bidi: isolate;">docstring</span> على مستوى الدالة يشرح الغرض وكيفية الاستخدام. إضافة إلى قسمي «المعاملات» و«القيمة المُعادة»، قدمنا شكلًا مبسطًا للمعادلة، ووضعنا رابطًا لمصدرها من <span dir="ltr" style="unicode-bidi: isolate;">FAO</span>. هذا كله ضروري لتمكين الآخرين من فهم كيفية الحساب والتحقق منه لاحقًا.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في قسم «المعاملات»، تم تحديد وحدات القياس المطلوبة لكل مدخل. هذا مهم جدًا؛ فمثلًا إذا قُدمت <span dir="ltr" style="unicode-bidi: isolate;">latitude</span> بوحدة الراديان بدل الدرجات ستكون النتيجة مختلفة وخاطئة. كما تم توضيح نوع البيانات في <span dir="ltr" style="unicode-bidi: isolate;">Python</span> مثل <span dir="ltr" style="unicode-bidi: isolate;">float</span>، لأن أخطاء نوع البيانات قد يصعب اكتشافها.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تم اختيار أسماء المتغيرات بعناية؛ نستخدم <span dir="ltr" style="unicode-bidi: isolate;">latitude</span> بدل اسم عام مثل <span dir="ltr" style="unicode-bidi: isolate;">x</span>. كما ميزنا بين الدرجات والراديان باستخدام <span dir="ltr" style="unicode-bidi: isolate;">latitude_radians</span>. وعند صعوبة تضمين الوحدة في الاسم، تساعد التعليقات داخل السطر في تتبع الوحدات مثلما فعلنا قرب <span dir="ltr" style="unicode-bidi: isolate;">solar_constant</span>.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تم تعريف الثوابت في أعلى الدالة مثل <span dir="ltr" style="unicode-bidi: isolate;">pi</span> و<span dir="ltr" style="unicode-bidi: isolate;">solar_constant</span>. تعريفها كمتغيرات يجعل المعنى أوضح ويتيح التحكم بالدقة من مكان واحد. عمومًا، يجب تعريف الثوابت مرة واحدة فقط.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">استخدمنا التعليقات بكثرة، خصوصًا قبل خطوات حساب <span dir="ltr" style="unicode-bidi: isolate;">sunset_hour_angle</span> لأنها أكثر تعقيدًا. كما أن أسماء المتغيرات التي تبدأ بشرطة سفلية مثل <span dir="ltr" style="unicode-bidi: isolate;">_hour_angle</span> تشير في <span dir="ltr" style="unicode-bidi: isolate;">Python</span> إلى أنها متغيرات وسيطة أقل أهمية.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في الحسابات الطويلة، قد يكون من المفيد تقسيم التعبير إلى كميات وسيطة ذات معنى مثل <span dir="ltr" style="unicode-bidi: isolate;">inc_radiation</span>. وإذا تعذّر ذلك، يساعد تقسيم التعبير عبر عدة أسطر في تحسين قابلية القراءة.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وأخيرًا، أضفنا <span dir="ltr" style="unicode-bidi: isolate;">assertions</span> باستخدام كلمة <span dir="ltr" style="unicode-bidi: isolate;">assert</span> لضمان استخدام الدالة بشكل صحيح. لاحظ ما يحدث عند تمرير نوع بيانات خاطئ أو قيمة خارج المجال لمدخل <span dir="ltr" style="unicode-bidi: isolate;">doy</span>.</p>
</div>

In [ ]:
toa_radiation(36.1, doy = 14.0)

In [ ]:
toa_radiation(36.1, doy = 500)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تحدٍّ</strong>: كتابة دالة موثقة توثيقًا جيدًا</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد أن راجعنا ما الذي يجعل الدالة موثقة بشكل جيد، اكتب دالة الخطوة التالية من التحليل. يمكن استخدام المعادلة التالية لحساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span>. اكتب دالة <span dir="ltr" style="unicode-bidi: isolate;">Python</span> موثقة باسم <span dir="ltr" style="unicode-bidi: isolate;">potential_et()</span> تُعيد <span dir="ltr" style="unicode-bidi: isolate;">PET</span> بوحدة مليمتر في اليوم <span dir="ltr" style="unicode-bidi: isolate;">(mm day^{-1})</span>.</p>
$$
\text{PET} = 0.0023 \times R_A \times \sqrt{T_{max} - T_{min}} \times (T + 17.8)
$$
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مدخلات الدالة <span dir="ltr" style="unicode-bidi: isolate;">potential_et()</span> هي:</p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">$R_A$ إشعاع الشمس أعلى الغلاف الجوي، بوحدة مكافئ ماء شهري <span dir="ltr" style="unicode-bidi: isolate;">(mm H2O equivalent per month)</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">$T_{max}$ درجة الحرارة العظمى الشهرية (درجات مئوية).</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">$T_{min}$ درجة الحرارة الصغرى الشهرية (درجات مئوية).</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">$T$ متوسط درجة الحرارة الشهرية (درجات مئوية).</li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تلميح:</strong> توجد دالة <span dir="ltr" style="unicode-bidi: isolate;">np.sqrt()</span> لحساب الجذر التربيعي.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">ابدأ بتعديل الدالة أدناه.</p>
</div>

In [ ]:
def potential_et(toa_radiation, temp_max, temp_min, temp_mean):
    pass

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">وسّع الخلية أدناه للاطلاع على أحد الحلول الممكنة.</p>
</div>

In [ ]:
def potential_et(toa_radiation, temp_max, temp_min, temp_mean):
    '''
    Calculates potential evapotranspiration, according to the Hargreaves
    equation:

    PET = 0.0023 * R * sqrt(Tmax - Tmin) * (Tmean + 17.8)

    Where R is the top-of-atmosphere (TOA) radiation (mm month-1); Tmax and 
    Tmin are the maximum and minimum monthly air temperatures (degrees C),
    respectively; and Tmean is monthly mean air temperature (degrees C).

    Parameters
    ----------
    toa_radiation : Number
        The top-of-atmosphere (TOA) radiation (mm day-1)
    temp_max : Number
        Maximum monthly air temperature (degrees C)
    temp_min : Number
        Minimum monthly air temperature (degrees C)
    temp_mean : Number
        Average monthly air temperature (degrees C)

    Returns
    -------
    Number
        The potential evapotranspiration (PET) in [mm day-1]
    '''
    return 0.0023 * toa_radiation * np.sqrt(temp_max - temp_min) * (temp_mean + 17.8)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إذا كُتبت الدالة بشكل صحيح، فعند استدعائها بالمدخلات أدناه ينبغي أن تحصل على قيمة قريبة من <span dir="ltr" style="unicode-bidi: isolate;">3.1</span>.</p>
</div>

In [ ]:
potential_et(10, 30, 20, 25)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الدوال المتجهة <span dir="ltr" style="unicode-bidi: isolate;">(Vectorized functions)</span></h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لنراجع كيف نستدعي دوال <span dir="ltr" style="unicode-bidi: isolate;">Python</span> التي كتبناها.</p>
</div>

In [ ]:
toa_radiation(32, 200)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تذكر أنه بسبب تعامل <span dir="ltr" style="unicode-bidi: isolate;">NumPy</span> مع المصفوفات كأنها أعداد، يمكننا استدعاء الدالة <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation()</span> بتمرير مصفوفة كأحد المدخلات.</p>
</div>

In [ ]:
lats = np.array([22, 32, 42])

toa_radiation(lats, 200)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في علم الحاسوب، يُسمّى استخدام الدالة بهذه الطريقة «التوجيه المتجهي» <span dir="ltr" style="unicode-bidi: isolate;">(Vectorization)</span>. وتسمى الدوال التي تعمل مع عدد واحد كما في <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation(32, 200)</span> أو مع مصفوفة كما في <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation(lats, 200)</span> دوالًا متجهة <span dir="ltr" style="unicode-bidi: isolate;">(Vectorized functions)</span>. وبما أننا غالبًا نتعامل مع مصفوفات بيانات بدل أعداد مفردة، فإن الدوال المتجهة شديدة الأهمية.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">على سبيل المثال، يسمح لنا التوجيه المتجهي برسم إشعاع <span dir="ltr" style="unicode-bidi: isolate;">TOA</span> كدالة في نطاق من الأيام. تبقى قيمة خط العرض ثابتة لكل قيمة من <span dir="ltr" style="unicode-bidi: isolate;">doy</span>، فتبدو العمليات الرياضية كأنها ضرب/جمع عدد واحد (خط العرض) بمصفوفة (أيام السنة).</p>
</div>

In [ ]:
from matplotlib import pyplot

doy = np.arange(1, 366).astype(np.int32)

rad = toa_radiation(32, doy)
pyplot.plot(doy, rad, 'k-')
pyplot.xlabel('Day of Year')
pyplot.ylabel('TOA Radiation [MJ m-2 day-1]')
pyplot.title('TOA Radiation at 32 deg N latitude')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>لكن إذا كان لدينا مصفوفتان كمدخلات أو أكثر، فيجب أن تكون الأشكال متوافقة.</strong> أدناه نحصل على خطأ لأننا نحاول ضرب مصفوفتين بأشكال غير متوافقة. فـ <span dir="ltr" style="unicode-bidi: isolate;">lats</span> تحتوي <span dir="ltr" style="unicode-bidi: isolate;">3</span> عناصر فقط، بينما <span dir="ltr" style="unicode-bidi: isolate;">doy</span> تحتوي <span dir="ltr" style="unicode-bidi: isolate;">365</span> عنصرًا.</p>
</div>

In [ ]:
# Won't work because array shapes are incompatible
toa_radiation(lats, doy)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الطريقة الوحيدة لإصلاح ذلك دون تغيير الدالة هي جعل المصفوفتين متوافقتين. يمكن إعادة تشكيلهما بحيث تصبحان مصفوفتين ثنائيتَي البعد <span dir="ltr" style="unicode-bidi: isolate;">T×N</span>، حيث <span dir="ltr" style="unicode-bidi: isolate;">T</span> هو عدد الأيام في <span dir="ltr" style="unicode-bidi: isolate;">doy</span> و<span dir="ltr" style="unicode-bidi: isolate;">N</span> هو عدد قيم خطوط العرض في <span dir="ltr" style="unicode-bidi: isolate;">lats</span>. هذا يعمل جيدًا، لكنه قد يكون مربكًا لأننا سنحصل على مصفوفة أكبر ثنائية الأبعاد.</p>
</div>

In [ ]:
toa_radiation(lats.reshape((1,3)), doy.reshape((365,1)))

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">اشتقاق متغيرات من إحداثيات <span dir="ltr" style="unicode-bidi: isolate;">xarray</span></h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سيكون التوجيه المتجهي أساسيًا لحساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> لبيانات درجات الحرارة الشبكية، لأن لدينا عدة خطوط عرض وعدة أيام في السنة يجب أخذها في الحسبان عند حساب إشعاع <span dir="ltr" style="unicode-bidi: isolate;">TOA</span>. انظر إلى إحداثيات مجموعة البيانات أدناه.</p>
</div>

In [ ]:
ds.coords

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>نحتاج إلى اشتقاق ما يلي:</strong></p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قيمة خط العرض لكل بكسل.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قيمة «اليوم من السنة» لكل بكسل.</li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تمثل مجموعة بيانات درجات الحرارة <span dir="ltr" style="unicode-bidi: isolate;">MERRA-2</span> عدد <span dir="ltr" style="unicode-bidi: isolate;">361</span> حزمة لخطوط العرض.</p>
</div>

In [ ]:
ds.lat.shape

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إحدى خطوات التوجيه المتجهي هي تحويل مصفوفة خطوط العرض أحادية البعد إلى «شبكة» من خطوط العرض، لأن كل بكسل يجب أن يمتلك قيمة خط عرض.</p>
</div>

In [ ]:
lats = ds['lat'].values
lats = lats.reshape((361, 1)).repeat(ds['lon'].size, axis = 1)
lats.shape

In [ ]:
pyplot.imshow(lats)
pyplot.colorbar()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تنبيه</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تبدو خطوط العرض لدينا مقلوبة في الشكل. إنشاء رسم كهذا طريقة ممتازة لفحص حدسنا.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>لكن هذا ليس مشكلة.</strong> تذكر أنه في <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> تتحرك إحداثيات <span dir="ltr" style="unicode-bidi: isolate;">lat</span> من <span dir="ltr" style="unicode-bidi: isolate;">-90</span> (جنوبًا) إلى <span dir="ltr" style="unicode-bidi: isolate;">+90</span> (شمالًا). نريد أن تطابق شبكة خطوط العرض ترتيب الإحداثيات في مجموعة البيانات لأن متغيرات البيانات نفسها منظمة بالطريقة ذاتها. يمكننا التحقق من ذلك عبر النظر إلى القيم الخام <span dir="ltr" style="unicode-bidi: isolate;">values</span> لأحد المتغيرات.</p>
</div>

In [ ]:
pyplot.imshow(ds['T2MMIN'].values[0])

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>وعليه، عند إضافة شبكة خطوط العرض إلى <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> كمتغير جديد، نحتاج لإخبار <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> بالمحاور الموافقة للشبكة.</strong> سنفعل ذلك بتعيين «صفّ» <span dir="ltr" style="unicode-bidi: isolate;">tuple</span> للمتغير الجديد <span dir="ltr" style="unicode-bidi: isolate;">"lat_grid"</span> كما يلي:</p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">العنصر الأول هو صف آخر <span dir="ltr" style="unicode-bidi: isolate;">('lat', 'lon')</span> يحدد ترتيب وأسماء المحاور، ويجب أن تكون هذه المحاور موجودة مسبقًا في <span dir="ltr" style="unicode-bidi: isolate;">ds</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">العنصر الثاني هو الشبكة نفسها <span dir="ltr" style="unicode-bidi: isolate;">lats</span>.</li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سيظهر المتغير الجديد <span dir="ltr" style="unicode-bidi: isolate;">"lat_grid"</span> بأبعاد <span dir="ltr" style="unicode-bidi: isolate;">(lat, lon)</span>.</p>
</div>

In [ ]:
ds['lat_grid'] = (('lat', 'lon'), lats)
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لكن متغيرات البيانات الأخرى (مثل <span dir="ltr" style="unicode-bidi: isolate;">T2MMIN</span> و<span dir="ltr" style="unicode-bidi: isolate;">T2MMAX</span>) تمتلك أبعادًا <span dir="ltr" style="unicode-bidi: isolate;">(time, lat, lon)</span>. سيكون تنفيذ بعض الحسابات لاحقًا أسهل بكثير إذا كانت <span dir="ltr" style="unicode-bidi: isolate;">"lat_grid"</span> تمتلك الأبعاد نفسها.</p>
</div>

In [ ]:
lats2 = lats.reshape((1, 361, 576)).repeat(122, axis = 0)
lats2.shape

In [ ]:
ds['lat_grid'] = (('time', 'lat', 'lon'), lats2)
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>الآن أصبح لدينا خط عرض لكل بكسل. ماذا عن «اليوم من السنة»؟</strong> لحسن الحظ يمكن اشتقاقه بسهولة من إحداثيات <span dir="ltr" style="unicode-bidi: isolate;">time</span> لأنها ممثلة بنوع بيانات <span dir="ltr" style="unicode-bidi: isolate;">datetime64[ns]</span> في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>، وبالتالي تمتلك مكونات تاريخ/وقت مثل <span dir="ltr" style="unicode-bidi: isolate;">month</span> و<span dir="ltr" style="unicode-bidi: isolate;">dayofyear</span>. يمكن قراءة المزيد هنا: <a href="https://docs.xarray.dev/en/stable/user-guide/time-series.html#datetime-components" dir="ltr">xarray datetime components</a>.</p>
</div>

In [ ]:
ds['time.dayofyear']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تطبيق دالة على كتل مستقلة</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بعد أن أصبح لدينا ما يلزم لحساب إشعاع <span dir="ltr" style="unicode-bidi: isolate;">TOA</span>، فلنختبر دالة <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation()</span> باستخدام بيانات تاريخ واحد.</p>
</div>

In [ ]:
test = ds.sel(time = '2024-05-01')

rad = toa_radiation(test['lat_grid'], test['time.dayofyear'])
rad

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">شكل النتيجة هو <span dir="ltr" style="unicode-bidi: isolate;">(1, 361, 576)</span> لأن المدخلين <span dir="ltr" style="unicode-bidi: isolate;">lat_grid</span> و<span dir="ltr" style="unicode-bidi: isolate;">time.dayofyear</span> لهما الشكل نفسه.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا إضافة هذه النتيجة بسهولة إلى <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> المسمّى <span dir="ltr" style="unicode-bidi: isolate;">test</span> عبر الإسناد.</p>
</div>

In [ ]:
test['toa_radiation'] = rad
test

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في هذه الحالة المتغير <span dir="ltr" style="unicode-bidi: isolate;">rad</span> هو أصلًا <span dir="ltr" style="unicode-bidi: isolate;">xarray.DataArray</span> بالأبعاد الصحيحة. ومع ذلك، من المفيد مراجعة الطريقة العامة لإضافة متغير بيانات إلى <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> عبر تحديد أسماء الأبعاد وترتيبها.</p>
</div>

In [ ]:
# NOTE: When rad is already an xarray.DataArray, we need to write rad.data
test['toa_radiation'] = (('time', 'lat', 'lon'), rad.data)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لنرسم النتيجة للتأكد من أنها منطقية. نرى أن إشعاع <span dir="ltr" style="unicode-bidi: isolate;">TOA</span> أعلى في نصف الكرة الشمالي حول نطاق عرض يقارب <span dir="ltr" style="unicode-bidi: isolate;">25</span> شمالًا، وهذا منطقي في هذا الوقت من السنة (مايو).</p>
</div>

In [ ]:
test['toa_radiation'].plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>كيف نطبق ما فعلناه على مجموعة البيانات كلها؟</strong> في الحقيقة، لم نكن بحاجة لاختيار يوم واحد كما فعلنا أعلاه.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لكن إذا كانت سلسلة الزمن طويلة جدًا، فقد نرغب في الاستفادة من المعالجة المتزامنة بشكل أفضل. فمشكلتنا (حساب إشعاع <span dir="ltr" style="unicode-bidi: isolate;">TOA</span>) مستقلة عبر المكان والزمان: النتيجة لا تعتمد على البكسلات المجاورة أو الخطوات الزمنية المجاورة. هذا النوع يُوصف غالبًا بأنه «قابل للتوازي بسهولة» <span dir="ltr" style="unicode-bidi: isolate;">(embarassingly parallel)</span> لأن عدد المهام المتوازية يمكن أن يكون كبيرًا جدًا دون التأثير على النتيجة.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قبل الاستفادة من المعالجة المتزامنة، فلنراجع كيف يقوم <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> بتقسيم البيانات إلى كتل حاليًا.</p>
</div>

In [ ]:
ds['lat_grid'].chunks

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يبدو أن متغير <span dir="ltr" style="unicode-bidi: isolate;">lat_grid</span> لا يمتلك كتلًا حتى الآن. لننشئها. أبسط نهج هو جعل كل خطوة زمنية (كل يوم) كتلة مستقلة. ويمكن ضبط بعدي <span dir="ltr" style="unicode-bidi: isolate;">lat</span> و<span dir="ltr" style="unicode-bidi: isolate;">lon</span> على <span dir="ltr" style="unicode-bidi: isolate;">'auto'</span> للإشارة إلى أننا لا نهتم بكيفية التقسيم على هذين البعدين.</p>
</div>

In [ ]:
# The size of the 'time' axis in each chunk should be 1: a different chunk for each time step
ds['lat_grid'] = ds['lat_grid'].chunk({'lat': 'auto', 'lon': 'auto', 'time': 1})
ds['lat_grid']

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إسناد دالة عامة عبر خريطة <span dir="ltr" style="unicode-bidi: isolate;">(Mapping)</span></h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أصبحنا الآن جاهزين لتطبيق دالة على الكتل المستقلة. في المعالجة المتزامنة يسمى ذلك «إسناد/تطبيق عبر خريطة» <span dir="ltr" style="unicode-bidi: isolate;">(mapping)</span> للدالة على أجزاء مستقلة من البيانات. في <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> تسمى هذه الأجزاء <strong>كتلًا</strong> <span dir="ltr" style="unicode-bidi: isolate;">(chunks)</span> أو <strong>بلوكات</strong> <span dir="ltr" style="unicode-bidi: isolate;">(blocks)</span>، ويُستخدم لهذا الغرض تابع <a href="https://docs.xarray.dev/en/stable/generated/xarray.map_blocks.html" dir="ltr">xarray.map_blocks()</a>.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تأمل المثال أدناه. لدينا دالة عامة <span dir="ltr" style="unicode-bidi: isolate;">my_function()</span> تأخذ <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> وتعيد نسخة معدلة من المتغير <span dir="ltr" style="unicode-bidi: isolate;">lat_grid</span>. عند تطبيقها على البلوكات المستقلة، نحصل على مكعب بيانات بالشكل والأبعاد نفسها.</p>
</div>

In [ ]:
# Trivial function that multiplies latitudes by two
def my_function(dataset):
    return dataset['lat_grid'] * 2

xr.map_blocks(my_function, ds)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مرة أخرى، لاحظ سرعة تنفيذ الشيفرة. تذكر أن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> يستخدم التقييم الكسول، ولم يحدث شيء فعلي بعد. نحتاج لاستدعاء <span dir="ltr" style="unicode-bidi: isolate;">compute()</span> لإخبار <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> أننا جاهزون لتحميل البيانات للذاكرة وتنفيذ الحساب.</p>
</div>

In [ ]:
result = xr.map_blocks(my_function, ds).compute()
result

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لنطبق الآن ذلك على دالة <span dir="ltr" style="unicode-bidi: isolate;">toa_radiation()</span>. هذه الدالة لا تعرف شيئًا عن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span>؛ فهي تفترض أننا نمرر أعدادًا أو مصفوفات <span dir="ltr" style="unicode-bidi: isolate;">numpy</span>. لذلك نحتاج أولًا إلى إنشاء دالة «مُغلِّفة» <span dir="ltr" style="unicode-bidi: isolate;">(wrapper)</span> كما في المثال التالي.</p>
</div>

In [ ]:
def toa_radiation_wrapper(dataset):
    return toa_radiation(dataset['lat_grid'], dataset['time.dayofyear'])

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مرة أخرى، نحتاج لإلحاق <span dir="ltr" style="unicode-bidi: isolate;">map_blocks()</span> باستدعاء <span dir="ltr" style="unicode-bidi: isolate;">compute()</span> عندما نكون جاهزين لتنفيذ الحساب.</p>
</div>

In [ ]:
result = xr.map_blocks(toa_radiation_wrapper, ds)

In [ ]:
toa_data = result.compute()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">إذا رسمنا الخطوة الزمنية نفسها كما في السابق، سنرى أننا حصلنا على النتيجة نفسها.</p>
</div>

In [ ]:
toa_data.sel(time = '2024-05-01').plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نحتاج أخيرًا إلى تحويل وحدات. يجب ضرب $R_A$ في <span dir="ltr" style="unicode-bidi: isolate;">0.408</span> لتحويله من <span dir="ltr" style="unicode-bidi: isolate;">[MJ m-2 day-1]</span> إلى <span dir="ltr" style="unicode-bidi: isolate;">[mm day-1]</span>. فلنحفظ هذا الناتج في <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> أيضًا، مع تسمية المتغير <span dir="ltr" style="unicode-bidi: isolate;">'toa_radiation'</span>.</p>
</div>

In [ ]:
# Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
ds['toa_radiation'] = toa_data * 0.408
ds

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أفضل ممارسة</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أصبح لدينا الآن كل البيانات اللازمة لحساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> على شبكة عالمية متسقة ضمن <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> واحد.</strong> لنتأكد من إضافة بعض البيانات الوصفية <span dir="ltr" style="unicode-bidi: isolate;">(metadata)</span> على مستوى الحقل، في حال شاركنا مجموعة البيانات هذه مع الآخرين.</p>
</div>

In [ ]:
ds['toa_radiation'].attrs

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لا توجد سمات <span dir="ltr" style="unicode-bidi: isolate;">attributes</span> حاليًا. ينبغي إضافة سمة واحدة على الأقل توضح وحدات القياس لمجال <span dir="ltr" style="unicode-bidi: isolate;">'toa_radiation'</span>.</p>
</div>

In [ ]:
ds['toa_radiation'].attrs['units'] = 'mm H2O day-1'
ds['toa_radiation'].attrs

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تحليل موارد الحوسبة <span dir="ltr" style="unicode-bidi: isolate;">(Profiling)</span></h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">دمج <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> و<span dir="ltr" style="unicode-bidi: isolate;">dask</span> يجعل استخدام المعالجة المتزامنة سهلًا. لكن الحواسيب لا تزال سريعة، وقد تكون هناك مهام كثيرة تُنفَّذ بسرعة على <span dir="ltr" style="unicode-bidi: isolate;">CPU</span> واحد حتى مع بيانات كبيرة، مما يجعل المعالجة المتزامنة غير ضرورية.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>أفضل ممارسة</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عمومًا ينبغي أن نجري «تحليلًا» <span dir="ltr" style="unicode-bidi: isolate;">profiling</span> لمتطلبات سير العمل قبل تنفيذ المعالجة المتزامنة أو غيرها من الحلول. كم يستغرق التنفيذ؟ وكم يستهلك من الذاكرة؟ من الأفضل الإجابة عن هذه الأسئلة أولًا بدل الافتراض.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في حالتنا، نستخدم مجموعة بيانات تعليمية صغيرة بما يكفي لتناسب الذاكرة. نعلم ذلك لأن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> يعرض حجم الذاكرة التي يحتاجها كل <span dir="ltr" style="unicode-bidi: isolate;">DataArray</span> عند فتح مجموعة البيانات. أدناه نرى أن متغيرًا واحدًا من <span dir="ltr" style="unicode-bidi: isolate;">MERRA-2</span> يحتاج نحو <span dir="ltr" style="unicode-bidi: isolate;">96.77 MB</span>.</p>
</div>

In [ ]:
ds['T2MMEAN'].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>مهمتنا ليست مقيدة بالذاكرة</strong> <span dir="ltr" style="unicode-bidi: isolate;">(memory-bound)</span>. لكن هل هي مقيدة بالمعالج <span dir="ltr" style="unicode-bidi: isolate;">(CPU-bound)</span>؟ إحدى الطرق للإجابة هي قياس الزمن على جزء صغير من البيانات. مثلًا، فلنجرب تنفيذ يوم واحد.</p>
</div>

In [ ]:
# Note there is exactly one chunk; i.e., the subsequent computation will not use more than one process
first_day = ds.sel(time = '2024-01-01')
first_day['T2MMEAN'].data

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أدناه أعدنا كتابة دالة <span dir="ltr" style="unicode-bidi: isolate;">potential_et()</span> بحيث تعمل مباشرة مع <span dir="ltr" style="unicode-bidi: isolate;">MERRA-2 xarray.Dataset</span>.</p>
</div>

In [ ]:
def potential_et(dataset):
    '''
    Calculates potential evapotranspiration, according to the Hargreaves
    equation:

    PET = 0.0023 * R * sqrt(Tmax - Tmin) * (Tmean + 17.8)

    Where R is the top-of-atmosphere (TOA) radiation (mm month-1); Tmax and 
    Tmin are the maximum and minimum monthly air temperatures (degrees C),
    respectively; and Tmean is monthly mean air temperature (degrees C).

    Single input argument should be an xarray.Dataset with the following
    data variables:

        T2MMIN: Maximum monthly air temperature (degrees C)
        T2MMAX: Minimum monthly air temperature (degrees C)
        T2MMEAN: Average monthly air temperature (degrees C)
        toa_radiation: The top-of-atmosphere (TOA) radiation (mm day-1)

    Parameters
    ----------
    dataset: xarray.Dataset

    Returns
    -------
    Number
        The potential evapotranspiration (PET) in [mm day-1]
    '''
    return 0.0023 * dataset['toa_radiation'] * np.sqrt(dataset['T2MMAX'] - dataset['T2MMIN']) * (dataset['T2MMEAN'] + 17.8)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قياس الزمن الحقيقي للمهمة باستخدام <span dir="ltr" style="unicode-bidi: isolate;">timeit</span></h3>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكننا استخدام وحدة <span dir="ltr" style="unicode-bidi: isolate;">timeit</span> المدمجة في <span dir="ltr" style="unicode-bidi: isolate;">Python</span> لقياس الزمن الذي تستغرقه كتلة شيفرة. سيُبلغنا <span dir="ltr" style="unicode-bidi: isolate;">timeit</span> عن «الزمن الحقيقي» <span dir="ltr" style="unicode-bidi: isolate;">(wall time)</span>، أي الزمن الذي يمضي على ساعة حائط أثناء تنفيذ المهمة. غالبًا هذا ما نهتم به عند تقدير مدة التنفيذ وما إذا كان من المفيد تقسيم المهمة عبر عمليات متزامنة متعددة.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لاستخدام <span dir="ltr" style="unicode-bidi: isolate;">timeit</span> داخل <span dir="ltr" style="unicode-bidi: isolate;">Jupyter Notebook</span>، نضيف الأمر السحري <span dir="ltr" style="unicode-bidi: isolate;">%%timeit</span> في أعلى خلية الشيفرة المراد قياسها.</p>
</div>

In [ ]:
%%timeit

potential_et(first_day)

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>لكن انتبه.</strong> تذكر أن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> يستخدم التقييم الكسول. نحتاج لإضافة <span dir="ltr" style="unicode-bidi: isolate;">compute()</span> للتأكد من أننا نقيس وقت المعالجة الفعلي.</p>
</div>

In [ ]:
%%timeit

potential_et(first_day).compute()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">سيقوم <span dir="ltr" style="unicode-bidi: isolate;">timeit</span> بتشغيل الخلية عدة مرات للحصول على متوسط الزمن. تختلف النتائج حسب الجهاز، لكن على أحد الأجهزة استغرق التنفيذ نحو <span dir="ltr" style="unicode-bidi: isolate;">20 ms</span>.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>إذا أردنا تقدير الزمن اللازم لتشغيل <span dir="ltr" style="unicode-bidi: isolate;">potential_et()</span> على مجموعة البيانات كاملة باستخدام عملية واحدة</strong>، يمكننا ضرب عدد الكتل (عدد خطوات الزمن) في زمن تنفيذ كتلة واحدة <span dir="ltr" style="unicode-bidi: isolate;">first_day</span>.</p>
</div>

In [ ]:
20e-3 * ds.time.size

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أي أن الزمن سيكون نحو <span dir="ltr" style="unicode-bidi: isolate;">2.4</span> ثانية فقط. وبما أننا نستخدم مجموعة بيانات أصغر لأغراض تعليمية، وبما أن <span dir="ltr" style="unicode-bidi: isolate;">potential_et()</span> دالة بسيطة نسبيًا، فستكون معالجة المجموعة كاملة سريعة.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">فلنشغل <span dir="ltr" style="unicode-bidi: isolate;">potential_et()</span> على مجموعة البيانات كاملة.</p>
</div>

In [ ]:
%%timeit

potential_et(ds).compute()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">في الواقع، استغرق تشغيل <span dir="ltr" style="unicode-bidi: isolate;">potential_et()</span> على مجموعة البيانات كاملة وقتًا أقل بكثير من <span dir="ltr" style="unicode-bidi: isolate;">2.4</span> ثانية. لماذا؟ لأن تقديرنا المبني على كتلة واحدة غالبًا ما يكون مبالغًا فيه؛ إذ إن <span dir="ltr" style="unicode-bidi: isolate;">xarray</span> ومُفسّر <span dir="ltr" style="unicode-bidi: isolate;">Python</span> يمكنهما تحسين تنفيذ المهام المتكررة ضمن حلقة، وهو ما يحدث فعليًا عند تنفيذ الدالة على عدة كتل باستخدام <span dir="ltr" style="unicode-bidi: isolate;">CPU</span> واحد.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الآن لنقارن زمن التنفيذ على <span dir="ltr" style="unicode-bidi: isolate;">CPU</span> واحد بزمن التنفيذ عند استخدام عمليات متعددة أدناه.</p>
</div>

In [ ]:
%%timeit

xr.map_blocks(potential_et, ds).compute()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">قد تلاحظ أن المهمة أصبحت أبطأ. السبب هو أن استخدام عمليات متعددة <span dir="ltr" style="unicode-bidi: isolate;">(concurrent processes)</span> يأتي مع حمل إضافي <span dir="ltr" style="unicode-bidi: isolate;">(overhead)</span> يتعلق بجدولة المهام وجمع النتائج. وقد يشمل ذلك انتظار عمليات متعددة دورها للقراءة/الكتابة. عندما تكون المهمة نفسها سريعة أصلًا، يصبح هذا الحمل الإضافي مؤثرًا.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يزداد الحمل الإضافي أيضًا عندما يكون عدد الكتل كبيرًا لأن المجدول <span dir="ltr" style="unicode-bidi: isolate;">scheduler</span> سيبذل مجهودًا أكبر لتنسيق العمل. <strong>اختيار حجم الكتل توازن مهم.</strong> إذا كانت الكتل كبيرة جدًا، قد لا يتحسن زمن التنفيذ الحقيقي كثيرًا لأن عدد العمليات المستخدمة فعليًا أقل، كما قد تظهر مشاكل الذاكرة. وإذا كانت الكتل صغيرة جدًا، قد تنتظر العمليات البيانات باستمرار.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن قراءة المزيد عن حجم الكتل والمعالجة المتزامنة هنا: <a href="https://docs.xarray.dev/en/stable/user-guide/dask.html#chunking-and-performance" dir="ltr">xarray dask chunking and performance</a>.</p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>تنبيه</strong></p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>عند استخدام الأمر السحري <span dir="ltr" style="unicode-bidi: isolate;">%%timeit</span> داخل خلية الشيفرة، قد يتداخل ذلك مع إسناد المتغيرات.</strong> عمومًا، ينبغي استخدام <span dir="ltr" style="unicode-bidi: isolate;">%%timeit</span> للتقدير فقط. وإذا أردنا الحصول على ناتج الحساب، ننفذ الشيفرة في خلية بدون <span dir="ltr" style="unicode-bidi: isolate;">%%timeit</span>.</p>
</div>

In [ ]:
pet = potential_et(ds).compute()

# If we applied a custom function, the result might inherit its name from
#   an existing input variable; we can reset the name this way
pet.name = 'Potential ET (mm day-1)'
pet.sel(time = '2024-01-01').plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن قراءة المزيد عن وحدة <span dir="ltr" style="unicode-bidi: isolate;">timeit</span> هنا:</p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://docs.python.org/3/library/timeit.html" dir="ltr">Python timeit documentation</a></li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><a href="https://sjvrijn.github.io/2019/09/28/how-to-timeit.html" dir="ltr">How to timeit (tutorial)</a></li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">استنتاج الجفاف الهيدرولوجي</h2>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أصبحنا جاهزين لاستخدام حساب <span dir="ltr" style="unicode-bidi: isolate;">PET</span> لدراسة الجفاف في تيارت.</p>
</div>

In [ ]:
# Subset the global PET data to Tiaret
pet_tiaret = pet.sel(lon = -1.32, lat = 35.37, method = 'nearest')
pet_tiaret

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">توضح السلسلة الزمنية لـ <span dir="ltr" style="unicode-bidi: isolate;">PET</span> كيف تغير فقد الماء إلى الغلاف الجوي. نرى أن <span dir="ltr" style="unicode-bidi: isolate;">PET</span> ازداد خلال هذه الفترة، لكن هذا التغير يشمل أيضًا تغيرات موسمية متوقعة نتيجة تغير ميلان الأرض.</p>
</div>

In [ ]:
pet_tiaret.plot()

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">لحساب مؤشر الجفاف الهيدرولوجي (نسبة الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">PET</span>) نحتاج إلى تحميل بيانات الهطول من <span dir="ltr" style="unicode-bidi: isolate;">CHIRPS</span> مرة أخرى.</p>
</div>

In [ ]:
chirps = xr.open_mfdataset('data_raw/CHIRPS/CHIRPS-v2_Africa_monthly_2014-2024.nc')

# Subset the CHIRPS data to Tiaret
chirps_tiaret = chirps['precip'].sel(x = slice(0.8, 1.8), y = slice(36.1, 35.1))
chirps_tiaret

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">نريد دمج مجموعتي البيانات، لكن:</p>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بيانات هطول <span dir="ltr" style="unicode-bidi: isolate;">CHIRPS</span> شهرية بوحدة <span dir="ltr" style="unicode-bidi: isolate;">(mm month^{-1})</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بيانات <span dir="ltr" style="unicode-bidi: isolate;">PET</span> التي حسبناها من <span dir="ltr" style="unicode-bidi: isolate;">MERRA-2</span> يومية بوحدة <span dir="ltr" style="unicode-bidi: isolate;">(mm day^{-1})</span>.</li>
</ul>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">كيف نعالج اختلاف الدقة الزمنية؟ يمكننا جمع <span dir="ltr" style="unicode-bidi: isolate;">PET</span> إلى خطوات شهرية، لكن الأفضل الحفاظ على دقته الزمنية الأعلى. لذلك سنقوم بإعادة أخذ عينات بيانات هطول <span dir="ltr" style="unicode-bidi: isolate;">CHIRPS</span> إلى خطوات يومية.</p>
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>سنفترض أن إجمالي الهطول الشهري موزع بالتساوي على أيام الشهر.</strong> يمكن فعل ذلك بسهولة باستخدام طريقة <span dir="ltr" style="unicode-bidi: isolate;">nearest()</span> التي تطبق استيفاء «أقرب جار» عبر نسخ القيمة الشهرية لكل يوم.</p>
</div>

In [ ]:
# Increasing the frequency of our monthly dataset to daily using nearest-neighbor interpolation
chirps_tiaret_resampled = chirps_tiaret.isel(time = slice(120, 125)).resample(time = 'D').nearest()
chirps_tiaret_resampled

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;"><strong>لا ينبغي أن ننسى قسمة الناتج على <span dir="ltr" style="unicode-bidi: isolate;">30</span> يومًا للحصول على متوسط الهطول اليومي.</strong> هذا تقريب بسيط لأن بعض الشهور لا تحتوي بالضبط على <span dir="ltr" style="unicode-bidi: isolate;">30</span> يومًا.</p>
</div>

In [ ]:
chirps_tiaret_daily = chirps_tiaret_resampled.mean(['x', 'y']) / 30
chirps_tiaret_daily

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">أصبحنا الآن جاهزين لحساب نسبة الهطول إلى <span dir="ltr" style="unicode-bidi: isolate;">PET</span> لتيارت.</p>
</div>

In [ ]:
ratio = 100 * chirps_tiaret_daily.values / pet_tiaret.values

pyplot.figure(figsize = (12, 4))

# Use the time coordinates as the X-axis values
pyplot.plot(pet['time'].values, ratio, 'k-')

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<p style="direction: rtl; unicode-bidi: plaintext; text-align: right;">بحد ذاته، لا يخبرنا الرسم أعلاه مدى شدة الجفاف في تيارت. فمع أن الهطول خلال الأشهر الأخيرة عوّض أقل من <span dir="ltr" style="unicode-bidi: isolate;">5%</span> من فقد المياه، قد يكون ذلك جزءًا من الدورة الموسمية الطبيعية. نحن نعلم أن يناير إلى أبريل فترة أكثر رطوبة نسبيًا في تيارت، لكن يبقى السؤال: <strong>هل يمكننا مقارنة هذا العام بالأعوام السابقة؟</strong></p>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
<h2 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الخلاصة</h2>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عند قراءة مجموعة بيانات باستخدام <span dir="ltr" style="unicode-bidi: isolate;">xarray.open_mfdataset()</span>، فإن <span dir="ltr" style="unicode-bidi: isolate;">xarray.Dataset</span> الناتج يمثل البيانات ولا يحمّلها فورًا إلى الذاكرة. كما تُقسّم البيانات تلقائيًا إلى كتل <span dir="ltr" style="unicode-bidi: isolate;">(chunks)</span> وفق موارد الجهاز. ويمكن معالجة هذه الكتل بشكل مستقل باستخدام المعالجة المتزامنة، حيث تُنفَّذ مهمة واحدة على كل كتلة باستخدام معالج واحد أو عدة معالجات.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">تشمل الموارد الحوسبية: الذاكرة، وسرعة ساعة <span dir="ltr" style="unicode-bidi: isolate;">CPU</span>، وسرعة القراءة/الكتابة من نظام الملفات. وتسمى المهام المقيدة بهذه الموارد على الترتيب: <span dir="ltr" style="unicode-bidi: isolate;">memory-bound</span> و<span dir="ltr" style="unicode-bidi: isolate;">CPU-bound</span> و<span dir="ltr" style="unicode-bidi: isolate;">I/O-bound</span>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الدوال المتجهة <span dir="ltr" style="unicode-bidi: isolate;">(vectorized)</span> هي دوال مصممة للعمل مع مدخلات قد تكون عددًا واحدًا أو مصفوفة مثل <span dir="ltr" style="unicode-bidi: isolate;">numpy.ndarray</span> أو <span dir="ltr" style="unicode-bidi: isolate;">xarray.DataArray</span>، ما يساعد على توسيع التحليل لمجموعات بيانات كبيرة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">عند استخدام المعالجة المتزامنة، يجب اختيار حجم الكتل بعناية. كثرة الكتل الصغيرة قد تسبب حملًا إضافيًا كبيرًا وقد تجعل المهمة أبطأ مقارنة بتنفيذها على عملية واحدة.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">يمكن تطبيق دالة مخصصة على كتل مستقلة (تسمى أيضًا بلوكات) باستخدام <span dir="ltr" style="unicode-bidi: isolate;">xarray.map_blocks()</span>.</li>
</ul>
</div>

<div dir="rtl" style="direction: rtl; unicode-bidi: isolate; text-align: right;">
<hr>
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">موارد إضافية</h3>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">مقال ممتاز من <span dir="ltr" style="unicode-bidi: isolate;">NCAR</span> حول استخدام <span dir="ltr" style="unicode-bidi: isolate;">dask</span> لتوسيع تحليل البيانات: <a href="https://ncar.github.io/Xarray-Dask-ESDS-2024/notebooks/02-dask-intro.html" dir="ltr">Using dask to scale up your data analysis</a>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">الحوسبة المتوازية باستخدام <span dir="ltr" style="unicode-bidi: isolate;">dask</span>: <a href="https://docs.xarray.dev/en/stable/user-guide/dask.html" dir="ltr">xarray dask guide</a>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">درس تعليمي عن استخدام <span dir="ltr" style="unicode-bidi: isolate;">timeit</span>: <a href="https://sjvrijn.github.io/2019/09/28/how-to-timeit.html" dir="ltr">How to timeit</a>.</li>
</ul>
<h3 style="direction: rtl; unicode-bidi: plaintext; text-align: right;">المراجع</h3>
<ul style="direction: rtl; unicode-bidi: plaintext; text-align: right;">
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">Bohr, Mark. 2007. "A 30-year retrospective on Dennard's MOSFET scaling paper." <a href="https://www.eng.auburn.edu/~agrawvd/COURSE/READING/LOWP/Boh07.pdf" dir="ltr">Boh07.pdf</a>.</li>
<li style="direction: rtl; unicode-bidi: plaintext; text-align: right;">Moore, Gordon E. 1965. "Cramming more components onto integrated circuits" <span dir="ltr" style="unicode-bidi: isolate;">Electronics Magazine</span>.</li>
</ul>
</div>